# Prequisite

## Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import PIL
from PIL import Image
from IPython.display import display

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision
import torchvision.transforms as T
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report

In [2]:
torch.manual_seed(42)
np.random.seed(42)
cv2.setRNGSeed(42)

ROOT = '../../dataset/v1'

MODEL_CKP = {
    'effnet': '../../pretrained_ckp/efficientnet_b0.pth'
    , 'shufflenet': '../../pretrained_ckp/shufflenet_v2.pth'
    , 'mobilenet': '../../pretrained_ckp/mobilenet_v2.pth'
}

device = torch.device("mps")
print(device)

print("PyTorch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

mps
PyTorch version: 2.1.0
torchvision version: 0.16.0


## Class and Functions

### Crop Face Function

In [3]:
def crop_face(image_path, target_width=300, target_height=400):
    """
    Crop and resize the detected face in an image.

    Parameters:
    - image_path (str): The path to the input image file.
    - target_width (int, optional): The desired width of the output face image (default is 300 pixels).
    - target_height (int, optional): The desired height of the output face image (default is 400 pixels).

    Returns:
    - resized_rgb (numpy.ndarray): The cropped and resized face region as a NumPy array in Gray color format.
    - resized_gray (numpy.ndarray): The cropped and resized face region as a NumPy array in Gray color format.

    Raises:
    - ValueError: If the image cannot be loaded from the given path or if no faces are detected in the image.

    This function takes an image file, detects the face in the image, and resizes it to match the specified target 
    width and height while maintaining the aspect ratio. The result is returned as a NumPy array in RGB and Gray color format.

    Example usage:
    >>> original_image, gray_iamge = crop_face("your_image.jpg")
    """

    image = cv2.imread(image_path)
    image_rb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Search for faces on the image.
    face_cascade = cv2.CascadeClassifier("../../haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
    x, y, w, h = faces[0]

    # Adjust height, so it will create a 3:4 (width:height) ratio.
    exp_ratio = 3 / 4
    h = int(w / exp_ratio)

    # Adjust y, as a pre-caution if it 
    # being cropped below the forehead.
    y -= int((image.shape[0] / target_height) * 35)
    
    # Add padding for the height, as a pre-caution
    # if it being cropped below the forehead.
    if y + h > image.shape[0]:
        minus_y = y + h - image.shape[0]
        y -= minus_y

    image_cropped = image_rb[y:y+h, x:x+w]
    image_cropped_resized = cv2.resize(image_cropped, (target_width, target_height))
    
    resized_rgb = image_cropped_resized
    resized_gray = cv2.cvtColor(resized_rgb, cv2.COLOR_BGR2GRAY)

    return resized_rgb, resized_gray

### Custom Dataset Class

In [4]:
class CustomDataset(Dataset):
    """
    Custom PyTorch dataset for working with image data and labels stored in a DataFrame.

    Parameters:
    - dataframe (pandas.DataFrame): The DataFrame containing image file paths and corresponding labels.
    - root (str): The root directory where the image files are located.
    - transform (callable, optional): A function/transform to apply to the images (e.g., data augmentation).

    Attributes:
    - dataframe (pandas.DataFrame): The input DataFrame containing image file paths and labels.
    - root (str): The root directory where image files are stored.
    - transform (callable, optional): A function/transform to be applied to the images.

    Methods:
    - __len__(): Returns the number of samples in the dataset.
    - __getitem__(idx): Returns the image and label for the specified index.

    This dataset class is designed to work with image data stored in a DataFrame, where each row contains
    a file path to an image and its corresponding label. It allows for data loading, cropping, and optional
    data transformation using PyTorch's data loading utilities. 

    Example usage:
    >>> dataset = CustomDataset(dataframe, root_dir, transform=transforms.Compose([transforms.Resize(256), transforms.ToTensor()]))
    >>> dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    >>> for images, labels in dataloader:
    >>>     # Process the batch of images and labels
    """

    def __init__(
        self
        , dataframe
        , root
        , transform
        , target_width
        , target_height
    ):
        self.dataframe = dataframe
        self.root = root
        self.transform = transform

        self.target_width = target_width
        self.target_height = target_height

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.dataframe.iloc[idx, 0])
        image, _ = crop_face(img_path, target_width=self.target_width, target_height=self.target_height)
        image = Image.fromarray(image)
        
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform: 
            image = self.transform(image)

        return image, label

### FKGTask Class

In [5]:
class FKGTask:
    """
    Designed to handle and prepare data for classification tasks. 
    It provides methods to encode categorical labels, split data into training and validation sets, 
    and retrieve the encoded data, making it useful for machine learning tasks.

    Parameters:
    - face_side (str): The column name representing the face side in the dataset.
    - subtask (str): The column name representing the subtask or label in the dataset.
    - dataframe (pd.DataFrame): The Pandas DataFrame containing the encoded data.

    Attributes:
    - face_side (str): The column name representing the face side in the dataset.
    - subtask (str): The column name representing the subtask or label in the dataset.
    - dataframe (pd.DataFrame): The Pandas DataFrame containing the encoded data.
    - encoding_dict (dict): A dictionary that maps the original class labels to their encoded values.
    - X (pd.Series): The feature data (independent variable).
    - y (pd.Series): The target data (dependent variable).

    Methods:
    - __init__(self, face_side, subtask, data): Initializes an instance of the FKGTask class.
    - _get_dataframe(self, data, subtask): Encodes the categorical labels in the dataset and returns the encoded DataFrame and encoding dictionary.
    - _get_x_and_y(self): Extracts the feature and target data from the DataFrame.
    - get_train_test_split(self): Splits the data into training and validation sets and returns them as DataFrames.

    Example Usage:
    >>> fkg_task = FKGTask(face_side=`face_side`, subtask=`subtask`, data=my_data)
    >>> train_data, val_data = fkg_task.get_train_test_split()
    
    >>>  # Train a machine learning model using the train_data
    >>>  # Validate the model using the val_data

    Note:
    - This class is designed to work with Pandas DataFrames and assumes that the input data contains columns corresponding to the specified `face_side` and `subtask`.
    - It uses ordinal encoding to convert categorical labels into numerical values.
    - The class provides a convenient way to split the data into training and validation sets, maintaining the stratified distribution of the target variable.
    """

    def __init__(self, face_side, subtask, data):
        self.face_side = face_side
        self.subtask = subtask
        self.dataframe, self.encoding_dict = self._get_dataframe(data, self.subtask)
        self.X, self.y = self._get_x_and_y()

    def _get_dataframe(self, data, subtask):
        encoder = OrdinalEncoder()
        unique_values = data[subtask][subtask].unique().reshape(-1, 1)
        encoder.fit(unique_values)

        before_val = data[subtask][subtask]
        data[subtask][subtask] = encoder.transform(data[subtask][subtask].values.reshape(-1, 1))
        encoding_dict = {original_class: encoded_value for original_class, encoded_value in zip(data[subtask][subtask], before_val)}
        encoding_dict = {v: k for k, v in encoding_dict.items()}
        
        return data[subtask], encoding_dict

    def _get_x_and_y(self):
        X = self.dataframe[self.face_side]
        y = self.dataframe[self.subtask]

        return X, y

    def get_train_test_split(self):
        X_train, X_val, y_train, y_val = train_test_split(
            self.X, self.y
            , stratify=self.y
            , test_size=0.2
            , random_state=42
        )

        return pd.concat([X_train, y_train], axis=1), pd.concat([X_val, y_val], axis=1)

### Load Model Function

In [6]:
def load_model_checkpoint(model, ckp_path):
    """
    Load a PyTorch model from a checkpoint file.
    This function loads a pre-trained or saved PyTorch 
    model from a checkpoint file and returns the loaded model.

    Parameters:
    - model (torch.nn.Module): The PyTorch model to be loaded.
    - ckp_path (str): The path to the checkpoint file.

    Returns:
    - loaded_model (torch.nn.Module): The model loaded from the checkpoint.

    Example Usage:
    >>> # Load a pre-trained model from a checkpoint file
    >>> model = load_model_checkpoint(models.resnet18(pretrained=False), 'model_checkpoint.pth')

    Note:
    - Ensure that the model architecture in the checkpoint file matches the provided `model` argument.
    """
    temp_model = model
    checkpoint = torch.load(ckp_path)
    temp_model.load_state_dict(checkpoint)

    return temp_model

def get_models():
    """
    Get a dictionary of pre-trained models.
    This function loads and returns a dictionary of pre-trained PyTorch models, 
    including EfficientNet B0, ShuffleNet V2, and MobileNet V2.

    Returns:
    - model_dict (dict): A dictionary with model names as keys and the corresponding pre-trained models as values.

    Example Usage:
    >>> # Get a dictionary of pre-trained models
    >>> models_dict = get_models()
    >>> effnet_model = models_dict['effnet']
    >>> shufflenet_model = models_dict['shufflenet']
    >>> mobilenet_model = models_dict['mobilenet']

    Note:
    - Make sure to import the necessary PyTorch model modules from `torchvision.models`.
    """
    efficientnet_b0 = load_model_checkpoint(models.efficientnet_b0(pretrained=False), MODEL_CKP['effnet'])  
    shufflenet = load_model_checkpoint(models.shufflenet_v2_x1_0(pretrained=False), MODEL_CKP['shufflenet'])
    mobilenet_v2 = load_model_checkpoint(models.mobilenet_v2(pretrained=False), MODEL_CKP['mobilenet'])

    model_dict = {}
    model_dict['effnet'] = efficientnet_b0
    model_dict['shufflenet'] = shufflenet
    model_dict['mobilenet'] = mobilenet_v2

    return model_dict

### Lightning Module Class

In [7]:
class FKGLightningModuleV1(pl.LightningModule):
    """
    FKGLightningModuleV1 is a PyTorch Lightning Module for image classification tasks. It allows training and validation of
    models with different architectures, such as EfficientNet, MobileNet, and ShuffleNet for classification problems.

    Parameters:
    - num_classes (int): The number of classes for the classification task.
    - model_name (str): The name of the model architecture to use from the model_dict.
    - model_dict (dict): A dictionary containing pre-trained PyTorch models.

    Attributes:
    - num_classes (int): The number of classes for the classification task.
    - model_name (str): The name of the model architecture being used.
    - model_dict (dict): A dictionary containing pre-trained PyTorch models.
    - model (nn.Module): The neural network model loaded from model_dict.
    - criterion (nn.Module): The loss function, Cross-Entropy Loss, used for training.

    Methods:
    - _load_model(self): Loads the specified model architecture and adjusts it for the given task.
    - forward(self, x): Performs forward pass through the model.
    - configure_optimizers(self): Configures the optimizer for training.
    - training_step(self, batch, batch_idx): Defines a training step, including forward and loss calculation.
    - validation_step(self, batch, batch_idx): Defines a validation step, including forward and loss calculation.

    Example Usage:
    >>> # Create an instance of FKGLightningModuleV1
    >>> model = FKGLightningModuleV1(num_classes=10, model_name='effnet', model_dict=model_dict)

    >>> # Configure Lightning Trainer and train the model
    >>> trainer = pl.Trainer(gpus=1)
    >>> trainer.fit(model, train_dataloader, val_dataloader)

    Note:
    - This class is designed to work with PyTorch Lightning for efficient training and validation of image classification models.
    - It supports various model architectures, and you need to provide a `model_dict` containing pre-trained models.
    - Make sure to customize the model architecture for your specific classification task by modifying the `_load_model` method.
    """

    def __init__(
        self
        , num_classes
        , model_name
        , model_dict
        , kernel_size=(3, 3)
        , change_kernel_size=False
    ):
        super(FKGLightningModuleV1, self).__init__()

        self.num_classes = num_classes
        self.model_name = model_name
        self.model_dict = model_dict

        self.training_loss = []
        self.training_loss_outputs = []

        self.kernel_size = kernel_size
        self.change_kernel_size = change_kernel_size
        
        self.model = self._load_model()
        self.criterion = nn.CrossEntropyLoss()

    def _change_kernel_size(self, model, model_type, kernel_size):
        new_kernel_size = kernel_size
        modified_first_conv = False
    
        if model_type == 'shufflenet':
            for layer in model.children():
                if isinstance(layer, nn.Sequential):
                    for child_layer in layer:
                        if isinstance(child_layer, nn.Conv2d):
                            if not modified_first_conv:     
                                # Modify the kernel_size for the first Conv2d layer in the first Sequential block
                                child_layer.kernel_size = new_kernel_size
                                modified_first_conv = True
        
        if model_type == 'effnet':
            for layer in model.children():
                if isinstance(layer, nn.Sequential):
                    for child_layer in layer:
                        for grandchild_layer in child_layer:
                            if isinstance(grandchild_layer, nn.Conv2d):
                                if not modified_first_conv:     
                                    # Modify the kernel_size for the first Conv2d layer in the first Sequential block
                                    grandchild_layer.kernel_size = new_kernel_size
                                    modified_first_conv = True
        
        if model_type == 'mobilenet':
            for layer in model.children():
                if isinstance(layer, nn.Sequential):
                    for grandchild_layer in layer[0]:
                        if isinstance(grandchild_layer, nn.Conv2d):
                            if not modified_first_conv:     
                                # Modify the kernel_size for the first Conv2d layer in the first Sequential block
                                grandchild_layer.kernel_size = new_kernel_size
                                modified_first_conv = True
        
        return model

    def _load_model(self):
        temp_model = self.model_dict[self.model_name]
        
        if self.model_name in ['effnet', 'mobilenet']:
            num_feat = temp_model.classifier[-1].in_features
            
        if self.model_name in ['shufflenet', 'resnet']:
            num_feat = temp_model.fc.in_features

        model = nn.Sequential(*list(temp_model.children())[:-1])
        model.add_module('global_avg_pool', nn.AdaptiveAvgPool2d(1))
        model.add_module('flatten', nn.Flatten())
        model.add_module('fc', nn.Linear(num_feat, self.num_classes))

        if self.change_kernel_size:
            model = self._change_kernel_size(model, self.model_name, self.kernel_size)
        
        return model
        
    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)   

        self.training_loss_outputs.append(loss)
        return loss

    # https://github.com/Lightning-AI/lightning/discussions/17182
    def on_train_epoch_end(self):
        epoch_average = torch.stack(self.training_loss_outputs).mean()
        self.training_loss.append(epoch_average)
        self.training_loss_outputs.clear()
   
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        val_loss = self.criterion(outputs, labels)
        return val_loss

# Pre-process

## Data

In [8]:
df = pd.read_excel('../../dataset/df_cls.xlsx')
df.dropna()
df.head()

,nama,tampak_depan_root,full_depan,tampak_samping_root,full_samping,tampak_senyum_root,full_senyum,tipe,simetris,seimbang,transversal,profil,nasolabial,mentolabial,segaris,bukal,kurva,garis
0,aisyahi,tampak_depan,tampak_depan/aisyahi.jpg,tampak_samping,tampak_samping/aisyahi.jpg,tampak_senyum,tampak_senyum/aisyahi.jpg,doliko,asimetris,tidak,tidak,cekung,normal,lebar,ya,lebar,datar,tinggi
1,alfira,tampak_depan,tampak_depan/alfira.jpg,tampak_samping,tampak_samping/alfira.jpg,tampak_senyum,tampak_senyum/alfira.jpg,doliko,simetris,ya,ya,cembung,tajam,tumpul,ya,normal,konsonan,rendah
2,despasya,tampak_depan,tampak_depan/despasya.jpg,tampak_samping,tampak_samping/despasya.jpg,tampak_senyum,tampak_senyum/despasya.jpg,doliko,simetris,ya,tidak,cembung,tajam,tumpul,ya,lebar,datar,sedang
3,galuh,tampak_depan,tampak_depan/galuh.jpg,tampak_samping,tampak_samping/galuh.jpg,tampak_senyum,tampak_senyum/galuh.jpg,doliko,simetris,ya,ya,cekung,tajam,tumpul,ya,lebar,konsonan,tinggi
4,happy,tampak_depan,tampak_depan/happy.jpg,tampak_samping,tampak_samping/happy.jpg,tampak_senyum,tampak_senyum/happy.jpg,brachy,simetris,ya,tidak,cembung,normal,tumpul,tidak,normal,datar,rendah


In [9]:
TASKS = [
    'tipe', 'simetris', 'seimbang', 'transversal'
    , 'profil', 'nasolabial', 'mentolabial', 'segaris'
    , 'bukal', 'kurva', 'garis'
]
df_tasks_dict = {}

for task in TASKS:
    if task in ['tipe', 'seimbang', 'simetris', 'transversal']:
        df_tasks_dict[task] = df[['full_depan', task]]
    elif task in ['profil', 'nasolabial', 'mentolabial']:
        df_tasks_dict[task] = df[['full_samping', task]]
    else: df_tasks_dict[task] = df[['full_senyum', task]]

## Model

In [10]:
model_dict = get_models()

/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# Training

## Training Constants

In [11]:
# ----- PERMUTATION ----- #
mod_strings = ['effnet', 'mobilenet', 'shufflenet']
subtasks = ['tipe', 'simetris', 'transversal', 'seimbang']
resolutions = [[400, 300], [600, 450], [800, 600]]
kernel_size = [(3, 3), (4, 3)]
# ----------------------- #

In [12]:
# Trial - Edit here
epoch_used = 10
max_epoch_used = epoch_used + 1
max_epoch_used

11

In [13]:
def return_data_transform(desired_size):
    data_transform = T.Compose([
        T.Resize(desired_size),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Use the ImageNet mean and std
    ])

    return data_transform

In [14]:
# Test
data_transform = return_data_transform(tuple(resolutions[2]))
data_transform

Compose(
    Resize(size=(800, 600), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

## Preparations

In [15]:
# Change here
subtask = 'transversal'

In [16]:
task = FKGTask('full_depan', subtask, df_tasks_dict.copy())
train, val = task.get_train_test_split()

/var/folders/0z/40d0w36j79z901rcfq942hhh0000gn/T/ipykernel_2590/447050079.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[subtask][subtask] = encoder.transform(data[subtask][subtask].values.reshape(-1, 1))


In [17]:
task.encoding_dict

{'tidak': 0.0, 'ya': 1.0}

## Start Loop

In [18]:
# Dataframe related
df_model = []
df_resolutions = []
df_kernel = []

df_f1 = []
df_recall = []
df_precision = []
df_accuracy = []

In [19]:
# Model related
tr_models = []
trainers = []

In [20]:
for mod in mod_strings:
    for resol in resolutions:
        data_transform = return_data_transform(tuple(resol))
        
        train_dataset = CustomDataset(
            dataframe=train, root=ROOT, transform=data_transform
            , target_width=resol[1], target_height=resol[0]
        )
        train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
        
        val_dataset = CustomDataset(
            dataframe=val, root=ROOT, transform=data_transform
            , target_width=resol[1], target_height=resol[0]
        )
        val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=0)

        for kernel in kernel_size:
            # Train the model
            if kernel == (4, 3): 
                _temp_model = FKGLightningModuleV1(2, mod, model_dict, kernel_size=(4,3), change_kernel_size=True)
            else: 
                _temp_model = FKGLightningModuleV1(2, mod, model_dict)

            _temp_trainer = pl.Trainer(max_epochs=max_epoch_used, accelerator="gpu", devices="auto")
            _temp_trainer.fit(model=_temp_model, train_dataloaders=train_loader)

            df_model.append(mod)
            df_resolutions.append(resol)
            df_kernel.append(kernel)

            tr_models.append(_temp_model)
            trainers.append(_temp_trainer)

            # Test result
            _temp_model_to_test = _temp_model
            predicted_labels = []
            true_labels = []
        
            _temp_model_to_test.eval()
            _temp_model_to_test.to(device)

            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = _temp_model_to_test(inputs)
                    _, predicted = torch.max(outputs, 1)
                    
                    predicted_labels.extend(predicted.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())
                    
            rep = classification_report(true_labels, predicted_labels, zero_division=0, output_dict=True)
            df_f1.append(rep['weighted avg']['f1-score'])
            df_recall.append(rep['weighted avg']['recall'])
            df_precision.append(rep['weighted avg']['precision'])
            df_accuracy.append(rep['accuracy'])

            # Get training loss
            plt.plot([tensor.item() for tensor in _temp_model.training_loss], label='Training Loss')
            plt.title(f'{mod}_{resol}_{kernel}')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            
            plt.savefig(f'{mod}_{resol}_{kernel}')
            plt.clf()

            # Save the model
            torch.save(_temp_model.model.state_dict(), f'{mod}_{resol}_{kernel}.pth')
            print(f'Done for: {mod}_{resol}_{kernel}')

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 4.0 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.040    Total estimated model params size (MB)
/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, doe

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=11` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 4.0 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.040    Total estimated model params size (MB)


Done for: effnet_[400, 300]_(3, 3)


/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


...

/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ardhanidzaky/Documents/02 University/02d Thesis/code/skripsi_env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=11` reached.


Done for: shufflenet_[800, 600]_(4, 3)


<Figure size 640x480 with 0 Axes>

In [25]:
# Start: 22.37 WIB
# End: 23.16 WIB

# Result

In [26]:
df_model = []
df_resolutions = []
df_kernel = []

for mod in ['effnet', 'mobilenet', 'shufflenet']:
    for resol in [[300, 400], [450, 600], [600, 800]]:
        for kernel in [(3,3), (4,3)]:
            df_model.append(mod)
            df_resolutions.append(resol)
            df_kernel.append(kernel)

In [27]:
df_result = pd.DataFrame({
    'Model': df_model
    , 'Resolutions': df_resolutions
    , 'Kernel Size': df_kernel
    , 'F1-score': df_f1
    , 'Recall': df_recall
    , 'Precision': df_precision
    , 'Accuracy': df_accuracy
})

In [28]:
df_result

,Model,Resolutions,Kernel Size,F1-score,Recall,Precision,Accuracy
0,effnet,"[300, 400]","(3, 3)",0.516484,0.5,0.541667,0.5
1,effnet,"[300, 400]","(4, 3)",0.321212,0.3,0.400000,0.3
2,effnet,"[450, 600]","(3, 3)",0.400000,0.4,0.552381,0.4
3,effnet,"[450, 600]","(4, 3)",0.576471,0.7,0.490000,0.7
4,effnet,"[600, 800]","(3, 3)",0.600000,0.6,0.600000,0.6
5,effnet,"[600, 800]","(4, 3)",0.576471,0.7,0.490000,0.7
6,mobilenet,"[300, 400]","(3, 3)",0.709890,0.7,0.733333,0.7
7,mobilenet,"[300, 400]","(4, 3)",0.474747,0.5,0.812500,0.5
8,mobilenet,"[450, 600]","(3, 3)",0.680000,0.7,0.675000,0.7
9,mobilenet,"[450, 600]","(4, 3)",0.762500,0.8,0.844444,0.8
